In [16]:
import data.audio_dataset_v1 as data_utils
import models.placeholder_model as model_utils
from abstract_model import AbstractModel

import torch
import torch.nn as nn
from torch import optim

from argparse import Namespace
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

from ipywidgets import IntProgress

In [2]:
AudioDataset = data_utils.AudioDataset
PlaceHolder = model_utils.PlaceHolder

In [4]:
audio_dataset = AudioDataset(root_dir='/scratch/prs392/incubator/data/LibriSpeech/train-clean-360')

for i in range(len(audio_dataset)):
    emb, spec, j = audio_dataset[i]
    print(emb.shape, spec.shape, j)
    m = PlaceHolder()
    pred = m(emb)
    print("Embeddings shape: " + str(emb.shape))
    print("Expected Spectrogram shape: " + str(spec.shape))
    print("Predicted Spectrogram shape: " + str(pred.shape))
    if i == 0:
        break

torch.Size([6144]) torch.Size([1, 128, 199]) tensor(19)
Embeddings shape: torch.Size([6144])
Expected Spectrogram shape: torch.Size([1, 128, 199])
Predicted Spectrogram shape: torch.Size([1, 1, 128, 199])


In [5]:
data_paths = {}
data_paths['train'] = '/scratch/prs392/incubator/data/LibriSpeech/train-clean-360'
data_paths['val'] = '/scratch/prs392/incubator/data/LibriSpeech/dev-clean'
# data_paths['test'] = '/scratch/prs392/incubator/data/LibriSpeech/test-clean'

def optimizer(model_parameters, hparams):
    optimizer = optim.SGD(model_parameters, lr=hparams.lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, hparams.scheduler_epoch, hparams.scheduler_step_size)
    return [optimizer], [scheduler]

args = {
    'batch_size': 32, # Compulsory
    'lr': 0.0002, # Compulsory
    'scheduler_epoch': 3, # Compulsory
    'scheduler_step_size': 0.1 # Compulsory
}

hparams = Namespace(**args)

In [18]:
hparams.lr

0.0002

In [ ]:
pl.seed_everything(123)

model = AbstractModel(
    hparams=hparams,
    data_paths = data_paths, 
    dataset_model = AudioDataset,
    model = PlaceHolder(), 
    criterion = nn.MSELoss(),
    optimizer = optimizer
)

logger = TensorBoardLogger("./", name="test_optimization")

trainer = pl.Trainer(logger=logger, 
                     max_epochs = 10, 
                     check_val_every_n_epoch=1,
                     gpus=-1,
                     distributed_backend='dp',
                     fast_dev_run=False)     
trainer.fit(model)

Set SLURM handle signals.

  | Name      | Type        | Params
------------------------------------------
0 | model     | PlaceHolder | 156 M 
1 | criterion | MSELoss     | 0     
